## Refatorando intervalso de eventos em Sinais

In [12]:
import os 

PATH = '/media/davi/6A81-05CF/physionet.org/files/siena-scalp-eeg/'

FILES = []

for root, dirs, files in os.walk(PATH):
    for file in files:
        if 'Seizures-list' in file:
            FILES.append(os.path.join(root,file))

In [17]:
f = open(FILES[0], "r")
fl = [i for i in f]

In [80]:
def get_patterns(file):
    patterns = [
        'File name:', 
        'Registration start time:',
        'Registration end time:',
        'Seizure start time:',
        'Seizure end time:'
    ]

    keys = ['name','registration_start','registration_end','seizure_start','seizure_end']

    data = []

    for line in file:
        for idx in range(len(keys)):
            if patterns[idx] in line:
                d = {}
                d[keys[idx]]= line.replace(patterns[idx],'').strip()
                data.append(d)

    return data


def set_intervals(data):
    intervals = [cont for cont in range(len(data)) if 'name' in data[cont].keys()]
    
    intervals.append(None)

    data_ = []
    
    for cont in range(len(intervals)-1):
        data_.append(data[intervals[cont]:intervals[cont+1]])

    return data_


def dict_update(data):
    d = {}
    for i in data:
        d.update(i)
    return d


def pipeline(file):
    data = get_patterns(file)
    result = set_intervals(data)
    
    return [dict_update(data) for data in result]



In [81]:
data = pipeline(fl)

In [84]:
data[0]

{'name': 'PN00-1.edf',
 'registration_start': '19.39.33',
 'registration_end': '20.22.58',
 'seizure_start': '19.58.36',
 'seizure_end': '19.59.46'}

In [108]:
from datetime import datetime
import numpy as np


def split_time(label, data):
    return [int(x) for x in data[label].split('.')]

def set_hour(label, data):
    h, m, s = split_time(label, data)
    
    date = datetime.now()

    return date.replace(hour=h,minute=m,second=s,microsecond=0)




d = data[0]

registration_start = set_hour('registration_start', d)
registration_end = set_hour('registration_end', d)
seizure_start = set_hour('seizure_start', d)
seizure_end = set_hour('seizure_end', d)

total = registration_end - registration_start
seizure_end_sec = registration_end - seizure_end
seizure_start_sec = registration_end - seizure_start

d['total'] = int(total.total_seconds())
d['seizure_start_sec'] = int(seizure_start_sec.total_seconds())
d['seizure_end_sec'] = int(seizure_end_sec.total_seconds())

d

{'name': 'PN00-1.edf',
 'registration_start': '19.39.33',
 'registration_end': '20.22.58',
 'seizure_start': '19.58.36',
 'seizure_end': '19.59.46',
 'total': 2605,
 'seizure_start_sec': 1462,
 'seizure_end_sec': 1392}